In [ ]:
import pandas as pd

SEDOL_COL = "SEDOL"
COMPANY_NAME_COL = "companyName"
TICKER_COL = "tickerSymbol"
SECTOR_COL = "sector"
SUBSECTOR_COL = "subsector"
COUNTRY_COL = "ISOCode"

def generate_metadata_fundamental(df_metadata: pd.DataFrame, independent_var_name: str) -> dict:
    parts = independent_var_name.split("_")
    sedol = parts[0]
    attribute_name = " ".join(parts[1:])

    row = df_metadata.loc[df_metadata[SEDOL_COL] == sedol, [COMPANY_NAME_COL, TICKER_COL, SECTOR_COL, SUBSECTOR_COL, COUNTRY_COL]].iloc[0]
    company_name, company_ticker, sector, subsector, country = row

    return {
        "Variable Name": independent_var_name,
        "Company Name": company_name,
        "Company Ticker": company_ticker,
        "Sedol": sedol,
        "Sector": sector,
        "Subsector": subsector,
        "Country": country,
        "Attribute": attribute_name
    }

In [ ]:
def get_rolling_correlation(independent_var_name: str, series1: pd.Series, series2: pd.Series, window_size: int = 12) -> List[pd.DataFrame]:
    corr_series = series1.rolling(window_size).corr(series2)
    df_rolling_corr = pd.DataFrame(corr_series, columns=[independent_var_name])
    
    parts = independent_var_name.split("_")
    sedol = parts[0]
    attribute = " ".join(parts[1:])

    df_stats = pd.DataFrame({
        "SEDOL": [sedol],
        "Attribute": [attribute],
        "Quantile 25%": [corr_series.quantile(0.25)],
        "Mean": [corr_series.mean()],
        "Median": [corr_series.median()],
        "Quantile 75%": [corr_series.quantile(0.75)],
        "Latest Value": [corr_series.iloc[-1]]
    }, index=[independent_var_name])
    
    return [df_rolling_corr, df_stats]

In [ ]:
MIN_OBSERVATIONS = 5
WEEKS_PER_YEAR = 52

def get_correlation(df: pd.DataFrame, dependent_var_name: str, independent_var_name: str,
                    rolling_corr: bool = True, rolling_windows: List[int] = [1,3,5,7]) -> Dict[str, Dict]:
    df_corr = df[[dependent_var_name, independent_var_name]].replace([-np.inf, np.inf], np.nan).dropna()
    num_obs = len(df_corr)
    
    if num_obs < MIN_OBSERVATIONS:
        empty_df = pd.DataFrame()
        return {
            "Stats": {"Num Obs": num_obs},
            "Time Series": {f"{year}y Rolling Corr": empty_df for year in rolling_windows},
            "Stats": {f"{year}y Rolling Corr": empty_df for year in rolling_windows}
        }
    
    dict_result = {
        "Stats": {"Num Obs": num_obs},
        "Time Series": {},
        "Stats": {}
    }
    
    if rolling_corr:
        results = [get_rolling_correlation(independent_var_name, df_corr.iloc[:,0], df_corr.iloc[:,1], window_size=year*WEEKS_PER_YEAR) for year in rolling_windows]
        for year, (df_rolling_corr, df_stats) in zip(rolling_windows, results):
            dict_result["Time Series"][f"{year}y Rolling Corr"] = df_rolling_corr
            dict_result["Stats"][f"{year}y Rolling Corr"] = df_stats
    
    return dict_result

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict

CHUNK_SIZE = 2500

def run_rolling_corr_calc(df_universe: pd.DataFrame, df_dependent: pd.DataFrame, dependent_variable_name: str,
                          run_rolling_corr: bool = True, rolling_windows: List[int] = [1,3,5,7]) -> List[pd.DataFrame]:
    
    df_universe = df_universe.replace([-np.inf, np.inf], np.nan).dropna(how='all')
    df_dependent = df_dependent.replace([-np.inf, np.inf], np.nan).dropna(how='all')
    
    df_temp = pd.merge(df_dependent, df_universe, left_index=True, right_index=True)
    
    final_timeseries_list = []
    final_stats_list = []

    for start_ind in range(0, df_universe.shape[1], CHUNK_SIZE):
        end_ind = min(start_ind + CHUNK_SIZE, df_universe.shape[1])
        
        independent_variable_names = list(df_temp.columns[start_ind:end_ind])
        if dependent_variable_name in independent_variable_names:
            independent_variable_names.remove(dependent_variable_name)

        results = [get_correlation(df_temp, dependent_variable_name, independent_variable_name, 
                                   run_rolling_corr, rolling_windows) 
                   for independent_variable_name in independent_variable_names]
        
        for year in rolling_windows:
            df_results_timeseries = pd.concat([item["Time Series"][f"{year}y Rolling Corr"] for item in results], axis=1)
            df_results_timeseries['rolling_window'] = f"{year}y"
            df_results_timeseries = df_results_timeseries.dropna(how='all').reset_index()
            final_timeseries_list.append(df_results_timeseries)
            
            df_stats = pd.concat([item["Stats"][f"{year}y Rolling Corr"] for item in results])
            df_stats['rolling_window'] = f"{year}y"
            df_stats = df_stats.dropna(how='all')
            df_stats.index.name = "Variable Name"
            df_stats = df_stats.reset_index()
            final_stats_list.append(df_stats)

    final_timeseries = pd.concat(final_timeseries_list, axis=0)
    final_stats = pd.concat(final_stats_list, axis=0)

    return [final_timeseries, final_stats]

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.graph_objects as go
from plotly.colors import qualitative
from dependencies.rolling_correlation import run_rolling_corr_calc

# Caching function for data loading and processing
@st.cache_data
def load_and_process_data(universe_name, target_sedol, rolling_wind):
    df_universe = pd.read_parquet(f"data/MSCI Europe_timeseries.parquet")
    df_dependent = df_universe[[f"{target_sedol}_Total_1y_Return"]]
    independent_var_list = [col for col in df_universe.columns if "12m_Fwd_PE" in col or "Price_to_Book" in col]
    df_universe = df_universe[independent_var_list]
    return run_rolling_corr_calc(df_universe, df_dependent, f"{target_sedol}_Total_1y_Return", 
                                 run_rolling_corr=True, rolling_windows=[rolling_wind])

# Streamlit app
st.info("This page focuses on allowing the user to filter and analyze the selected dataset.")
st.subheader("Rolling Correlation")

# Load metadata
universe_name = st.session_state.universe_name
df_metadata = pd.read_csv(f"data/metadata/{universe_name}_constituent_list.csv", index_col=0)

# Company selection
company_list = list(df_metadata["tickerSymbol"] + " SEDOL=" + df_metadata["SEDOL"])
target_ticker = st.selectbox("Target Company Ticker (*required)", company_list, index=0)
target_sedol = target_ticker.split("=")[1]

# Rolling window selection
rolling_wind = st.selectbox("Select Rolling Window", [1, 3, 5, 7], index=3)

# Load and process data
df_timeseries, df_stats = load_and_process_data(universe_name, target_sedol, rolling_wind)

# Prepare stats dataframe
df_stats.index = df_stats["Variable Name"]
df_stats = df_stats.drop("Variable Name", axis=1)

# Time range selection
years_options = ['1 Year', '3 Years', '5 Years', '7 Years', 'Max']
selected_years = st.selectbox('Select the number of years:', years_options, index=4)

# Variable selection
selected_variables = st.multiselect("Select variables to plot", df_stats.index.tolist())

if selected_variables:
    st.subheader("Rolling Correlation Plot")
    
    # Prepare data for plotting
    df_plot = df_timeseries[selected_variables].set_index('pricingDate')
    df_plot = df_plot.dropna(how='all')

    # Apply time range filter
    if selected_years != 'Max':
        years = int(selected_years.split(' ')[0])
        latest_date = df_plot.index.max()
        start_date = latest_date - pd.DateOffset(years=years)
        df_plot = df_plot.loc[start_date:]

    # Create plot
    fig = go.Figure()
    color_scale = qualitative.Plotly

    for i, col in enumerate(df_plot.columns):
        attribute_info = col.split("_", 1)[1]
        fig.add_trace(go.Scatter(
            x=df_plot.index, 
            y=df_plot[col],
            mode='lines', 
            name=f'{target_ticker.split(" ")[0]} ({attribute_info})', 
            line=dict(width=4, color=color_scale[i % len(color_scale)])
        ))

    # Update layout
    fig.update_layout(
        title=f"{rolling_wind}-Year Rolling Correlation",
        xaxis_title="Date",
        yaxis_title="Correlation",
        legend_title="Variables",
        height=600
    )

    # Display plot
    st.plotly_chart(fig, use_container_width=True)

    # Display stats
    st.subheader("Correlation Statistics")
    st.dataframe(df_stats.loc[selected_variables], use_container_width=True)
else:
    st.warning("Please select at least one variable to plot.")